In [ ]:
from medigraph.data.abide import AbideData
import numpy as np
from nilearn import plotting
from medigraph.model.gcn import GCN, SparseGCN, ChebGCN
from medigraph.model.baseline import DenseNN
import torch
from tqdm.notebook import tqdm
from medigraph.data.preprocess import sanitize_data, visual_sanity_check_input, whiten
from medigraph.data.experiences_data import get_training_dict_exp1, get_training_dict_exp2
from medigraph.train import training_loop, train, plot_learning_curves, compute_metrics
from medigraph.data.properties import INPUTS, LABELS, TRAIN_MASK, VAL_MASK, TEST_MASK, ADJ
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%load_ext autoreload
%autoreload 2
device

In [ ]:
dat = AbideData()

### Check connectivy matrix for a single patient
- 111x111 matrices
- We'll retrieve the $6216=\frac{111*(111+1)}{2}$ raw coefficients from the upper triangular matrix

In [ ]:
# Get the connectivity matrix for the first subject
idx = 0
mat = dat.get_connectivity_matrix(idx)
plotting.plot_matrix(
    mat,
    figure=(6, 6),
    vmax=1,
    vmin=0,
    title=f"Patient {idx} connectivity matrix {mat.shape}"
)
feature_vector_input = dat.get_connectivity_features(idx)
print(f"input feature vector shape: {feature_vector_input.shape}")

# Train classifier

### Build adjacency, features matrix and classification labels

In [ ]:
# % Build adjacency matrix and input feature vectors
data_dict = dat.get_training_data(dimension_reduction="raw_inputs")
inp_np, lab_np, adj_np = data_dict[INPUTS], data_dict[LABELS], data_dict[ADJ]
print(f"Adjacency matrix : {adj_np.shape} [VxV]")
print(f"Labels {lab_np.shape} : [V]")
print(f"Input feature vector {inp_np.shape} : [VxF]")

In [ ]:
# % Load data to GPU
labels_np = dat.get_labels()
adj = torch.tensor(adj_np, dtype=torch.float32).to(device)
inp_raw = torch.tensor(inp_np, dtype=torch.float32).to(device)  # [V=871,  F6216]
lab = torch.tensor(labels_np, dtype=torch.float32).to(device)  # for binary classification

In [ ]:
# % Sanitize and whiten data
clean_inp = sanitize_data(inp_raw)
inp = whiten(clean_inp)
inp.shape, adj.shape, lab.shape

In [ ]:
# % Visalization of sanity check
visual_sanity_check_input(inp_raw)
visual_sanity_check_input(clean_inp)
visual_sanity_check_input(inp)

In [ ]:
# % sanity check on graph adjacency matrix
model = GCN(inp.shape[1], adj, hdim=64)
plotting.plot_matrix(
    model.adj.detach().cpu().numpy(),
    figure=(6, 6),
    vmax=0.005,
    vmin=0,
    title=f"Graph normalized adjacency matrix {mat.shape}"
)
del model

In [ ]:
training_data = {
    INPUTS: inp,
    LABELS: lab
}
metric_dict = {}
for model_name in ["Dense", "GCN"]:
    if model_name == "GCN":
        model = GCN(inp.shape[1], adj, hdim=64)
    else:
        model = DenseNN(inp.shape[1], hdim=64)
    model.to(device)

    model, metrics = training_loop(model, training_data, device, n_epochs=1000)
    metric_dict[model_name] = metrics

In [ ]:
def plot_metrics(metric_dict: dict):
    fig, axs = plt.subplots(1, 2, figsize=(10, 6))
    for model_name, metric in metric_dict.items():
        print(metric.keys())
        axs[0].plot(metric["training_losses"], label=model_name)
        axs[1].plot(metric["training_accuracies"], label=f"{model_name} accuracy")
    for ax in axs:
        ax.legend()
        ax.grid()
    axs[0].set_title("Training loss (Binary Cross Entropy)")
    axs[1].set_title("Accuracy")

    plt.show()


plot_metrics(metric_dict)

---

## Exp 1 : Train classifiers without reduction of features

Here we trained the classifier on all the features (using the 'sanitizer made by Balthazar')


In [ ]:
training_dict_exp1 = get_training_dict_exp1(dat, nb_train=700, nb_val=100, override=False)
# put to tensor and to device
training_dict_exp1[TRAIN_MASK] = torch.LongTensor(training_dict_exp1[TRAIN_MASK]).to(device)
training_dict_exp1[VAL_MASK] = torch.LongTensor(training_dict_exp1[VAL_MASK]).to(device)
training_dict_exp1[TEST_MASK] = torch.LongTensor(training_dict_exp1[TEST_MASK]).to(device)
training_dict_exp1[INPUTS] = torch.tensor(training_dict_exp1[INPUTS], dtype=torch.float32).to(device)
training_dict_exp1[LABELS] = torch.tensor(training_dict_exp1[LABELS], dtype=torch.float32).unsqueeze(1).to(device)


in_channels_exp1 = training_dict_exp1[INPUTS].shape[1]

Understand and visualize the feature (reflection to add to the report ?)

plot the correlation matrix, visualize with PCA or with UMAP ?

### Baseline : DenseNN

In [ ]:
DenseNN_exp1 = DenseNN(in_channels_exp1,
                       hdim=64).to(device)

In [ ]:
if training_dict_exp1[LABELS].ndim > 1:
    training_dict_exp1[LABELS] = training_dict_exp1[LABELS].squeeze(1)
DenseNN_exp1, DenseNN_exp1_train_log, DenseNN_exp1_val_log = train(DenseNN_exp1, 
                                                                   training_dict_exp1,
                                                                   nEpochs=200,
                                                                   optimizer_params={'lr': 0.001, 
                                                                                     'weight_decay': 0.05})

In [ ]:
plot_learning_curves(DenseNN_exp1_train_log, DenseNN_exp1_val_log, title="Training of DenseNN_exp1")

In [ ]:
# Testing 
loss_test, acc_test = compute_metrics(DenseNN_exp1, training_dict_exp1)
print(f"Test loss: {loss_test.item():.4f}, Test accuracy: {acc_test.item():.4f}")

### GCN 

In [ ]:
if training_dict_exp1[LABELS].ndim == 1:
    training_dict_exp1[LABELS] = training_dict_exp1[LABELS].unsqueeze(1)

GCN_exp1 = SparseGCN(in_channels_exp1, 
                  nhid=[256, 128], 
                  nclass=1, 
                  adjacency=training_dict_exp1[ADJ], 
                  proba_dropout=0.6,
                  device=device).to(device)

In [ ]:
GCN_exp1, GCN_exp1_train_log, GCN_exp1_val_log = train(GCN_exp1, training_dict_exp1, 
                                              nEpochs=200, 
                                              optimizer_params={'lr': 0.01, 'weight_decay': 0.05})
# torch.save(GCN_exp1.state_dict(), "__GCN_exp1")

In [ ]:
plot_learning_curves(GCN_exp1_train_log, GCN_exp1_val_log, title="Training of GCN_exp1")

In [ ]:
# Testing 
loss_test, acc_test = compute_metrics(GCN_exp1, training_dict_exp1)
print(f"Test loss: {loss_test.item():.4f}, Test accuracy: {acc_test.item():.4f}")

### ChebGCN on exp1
TO DO

---

## Exp 2 : Train classifiers with selected feat

Here, we train the same network with features reduced by a ridge classifier

In [ ]:
training_dict_exp2 = get_training_dict_exp2(dat, nb_train=700, nb_val=100, override=False)
# put to tensor and to device
training_dict_exp2[TRAIN_MASK] = torch.LongTensor(training_dict_exp2[TRAIN_MASK]).to(device)
training_dict_exp2[VAL_MASK] = torch.LongTensor(training_dict_exp2[VAL_MASK]).to(device)
training_dict_exp2[TEST_MASK] = torch.LongTensor(training_dict_exp2[TEST_MASK]).to(device)
training_dict_exp2[INPUTS] = torch.tensor(training_dict_exp2[INPUTS], dtype=torch.float32).to(device)
# training_dict_exp2[LABELS] = torch.tensor(training_dict_exp2[LABELS], dtype=torch.float32).unsqueeze(1).to(device)

training_dict_exp2[LABELS] = torch.tensor(training_dict_exp2[LABELS], dtype=torch.float32).to(device)

in_channels_exp2 = training_dict_exp2[INPUTS].shape[1]

Understand and visualize the data to compare with the precedent experience 

Correlation matrix, PCA, UMAP ? 

In [ ]:
# PCA : TO DO

# selected_feat = get_training_dict_exp2[INPUTS]
# print(selected_feat.shape)

### Baseline : DenseNN

In [ ]:
DenseNN_exp2 = DenseNN(in_channels_exp2,
                       hdim=64).to(device)

In [ ]:
if training_dict_exp2[LABELS].ndim > 1:
    training_dict_exp2[LABELS] = training_dict_exp2[LABELS].squeeze(1)
DenseNN_exp2, DenseNN_exp2_train_log, DenseNN_exp2_val_log = train(DenseNN_exp2, 
                                                                   training_dict_exp2,
                                                                   nEpochs=200,
                                                                   optimizer_params={'lr': 0.001, 
                                                                                     'weight_decay': 0.05})

In [ ]:
plot_learning_curves(DenseNN_exp2_train_log, DenseNN_exp2_val_log, title="Training of DenseNN_exp2")

In [ ]:
# Testing 
loss_test, acc_test = compute_metrics(DenseNN_exp2, training_dict_exp2)
print(f"Test loss: {loss_test.item():.4f}, Test accuracy: {acc_test.item():.4f}")

### GCN

In [ ]:
if training_dict_exp2[LABELS].ndim == 1:
    training_dict_exp2[LABELS] = training_dict_exp2[LABELS].unsqueeze(1)

GCN_exp2 = SparseGCN(in_channels_exp2, 
                  nhid=[128, 64], 
                  nclass=1, 
                  adjacency=training_dict_exp2[ADJ], 
                  proba_dropout=0.3).to(device)

In [ ]:
# Testing  before training to see if the model get better
loss_test, acc_test = compute_metrics(GCN_exp2, training_dict_exp2)
print(f"Test Before Training \n loss: {loss_test.item():.4f}, Test accuracy: {acc_test.item():.4f}")

In [ ]:
# training_dict_exp2[LABELS] = training_dict_exp2[LABELS].squeeze(1)
GCN_exp2, GCN_exp2_train_log, GCN_exp2_val_log = train(GCN_exp2, training_dict_exp2, 
                                              nEpochs=200, 
                                              optimizer_params={'lr': 0.001, 'weight_decay': 0.005})
# torch.save(trained_model_GCN.state_dict(), "__trained_model_GCN")

In [ ]:
plot_learning_curves(GCN_exp2_train_log, GCN_exp2_val_log, title="Training of GCN_exp2")

In [ ]:
np.unique(GCN_exp2_val_log["accuracies"])

In [ ]:
# Testing after training
loss_test, acc_test = compute_metrics(GCN_exp2, training_dict_exp2)
print(f"Test loss: {loss_test.item():.4f}, Test accuracy: {acc_test.item():.4f}")

### ChebConv

In [ ]:
ChebGCN_exp2 = ChebGCN(in_channels_exp2, out_features=1, K=3,
                       adjacency=training_dict_exp2[ADJ],
                       proba_dropout=0.3, device=device).to(device)

In [ ]:
ChebGCN_exp2, ChebGCN_exp2_train_log, ChebGCN_exp2_val_log = train(ChebGCN_exp2, training_dict_exp2,
                                            nEpochs=200,
                                            optimizer_params={'lr': 0.005, 'weight_decay': 0.005})

In [ ]:
plot_learning_curves(ChebGCN_exp2_train_log, ChebGCN_exp2_val_log, title="Training of ChebGCN_exp2")

In [ ]:
# Testing
loss_test, acc_test = test(ChebGCN_exp2, training_dict_exp2)
print(f"Test loss: {loss_test.item():.4f}, Test accuracy: {acc_test.item():.4f}")